<a href="https://colab.research.google.com/github/jjuliacp/jjuliacp/blob/main/Copia_de_SQL_Murder_Mystery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/icarito/UPSK-SQL001-SQLZoo-murder/blob/sql-murder/Sql-Murder-Mystery/SQL_Murder_Mystery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL Murder Mystery!
Adapted By: Martin Arroyo

![Detective making connections between points](https://github.com/freestackinitiative/coop_sql_notebooks/blob/main/assets/sleuth.png?raw=1)

**Credit**

This material was adapted from the [SQL Murder Mystery by Knight Lab](https://mystery.knightlab.com/) under [Creative Commons CC BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/). The SQL Murder Mystery was originally created by [Joon Park](https://twitter.com/joonparkmusic) and [Cathy He](https://twitter.com/Cathy_MeiyingHe) while they were Knight Lab fellows. See the [GitHub repository](https://github.com/NUKnightLab/sql-mysteries) for more information.

## Escenario

¡Se ha cometido un crimen y los detectives necesitan tu ayuda! Te dieron el informe de la escena del crimen, pero de alguna manera lo perdiste. Recuerdas vagamente que el crimen fue un `murder` _(asesinato)_ que ocurrió en algún momento el 15 de Enero de 2018 y que tuvo lugar en `SQL City`. Depende de ti averiguar quién es el asesino usando solo tus habilidades en SQL y tu ingenio. Te proporcionan una conexión a la base de datos del Departamento de Policía, que tiene todas las pistas que necesitarás para atrapar al asesino.

Usa las habilidades que desarrollaste en SQL 101 y 102, junto con cualquier recurso que desees, para resolver el `SQL Murder Mystery`!"

## Conéctate a la base de datos del Departamento de Policía

Para comenzar y ejecutar tus consultas, presiona play en la celda de abajo para conectarte a la base de datos del Departamento de Policía.

Para ejecutar consultas, crea una nueva celda de `Code` y escribe `%%sql` en la parte superior. Luego puedes escribir tus consultas debajo. Ve el ejemplo a continuación:
```python
%%sql

SELECT *
FROM table

```

In [1]:
%%capture
# @title Press Play { display-mode: "form" }
# Install `teachdb` and `coop_grader`
print("Installing `teachdb` and its dependencies...")
!pip install --quiet --upgrade git+https://github.com/freestackinitiative/teachingdb.git git+https://github.com/martinmarroyo/coop_grader.git
print("Successfully installed `teachdb`")
import pandas as pd
from teachdb.teachdb import connect_teachdb
from coop_grader.sql_murder_mystery.check_suspect import check_suspect
# Set configurations for notebook
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 99)
# Load data
con = connect_teachdb(database="sql_murder_mystery")

%sql con

## Descubriendo tablas en la base de datos

Comenzamos nuestra búsqueda para encontrar al asesino explorando la base de datos del Departamento de Policía. Pero aún no has visto la base de datos y no sabes cuáles son las tablas, así que, ¿cómo sabes qué buscar?

Afortunadamente, la mayoría de los sistemas de gestión de bases de datos relacionales tienen esta información almacenada en un lugar donde puedes consultarla. Muy a menudo, se utiliza un esquema especial conocido como [`information_schema`](https://en.wikipedia.org/wiki/Information_schema) para almacenar información sobre las tablas y columnas en tu base de datos (también conocido como metadata). La base de datos del Departamento de Policía tiene un information schema, con la vista `tables` que te muestra qué tablas están disponibles, y la vista `columns` que te muestra todas las columnas de cada tabla y sus tipos de datos.

### Listando todas las tablas en la base de datos de la Policía

#### Revisa la [descripción de `information_schema.tables`](https://duckdb.org/docs/sql/information_schema.html#tables-and-views)

Primero, veremos todas las tablas disponibles para nosotros consultando la vista `information_schema.tables`. Te daremos la primera consulta para empezar, pero de aquí en adelante tendrás que idear las consultas restantes utilizando tu conocimiento de SQL y tu ingenio.

Aquí está la consulta necesaria para mostrarte las tablas en la base de datos del Departamento de Policía. Cópiala/Pégala en la celda de abajo y ejecútala para ver las tablas disponibles para ti:

>```sql
>SELECT *
>FROM information_schema.tables
>```

In [ ]:
%%sql
SELECT *
FROM information_schema.tables

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action,TABLE_COMMENT
0,memory,main,crime_scene_report,BASE TABLE,None,None,None,None,None,YES,NO,None,None
1,memory,main,drivers_license,BASE TABLE,None,None,None,None,None,YES,NO,None,None
2,memory,main,facebook_event_checkin,BASE TABLE,None,None,None,None,None,YES,NO,None,None
3,memory,main,get_fit_now_check_in,BASE TABLE,None,None,None,None,None,YES,NO,None,None
4,memory,main,get_fit_now_member,BASE TABLE,None,None,None,None,None,YES,NO,None,None
5,memory,main,income,BASE TABLE,None,None,None,None,None,YES,NO,None,None
6,memory,main,interview,BASE TABLE,None,None,None,None,None,YES,NO,None,None
7,memory,main,person,BASE TABLE,None,None,None,None,None,YES,NO,None,None


### Listando todas las tablas y sus columnas en la base de datos de la Policía

**Revisa la [descripción de `information_schema.columns`](https://duckdb.org/docs/sql/information_schema.html#columns)**

¡Genial! Ahora que sabes qué tablas están disponibles, es hora de averiguar las columnas que tiene cada tabla junto con el tipo de datos de cada columna. Escribe una consulta que muestre el nombre de la tabla, el nombre de la columna y el tipo de datos para cada tabla en la base de datos del Departamento de Policía utilizando la vista `information_schema.columns`. Asegúrate de que la salida esté ordenada por nombre de tabla y luego por nombre de columna (ascendente):

*Pistas opcionales - ¡intenta usarlas solo si te quedas **REALMENTE** atascado!*

<details>
<summary>Pista 1</summary>
<p>Estructura tu consulta de la misma manera que lo hicimos en la consulta anterior donde miramos `information_schema.tables`</p>
</details>

<details>
<summary>Pista 2</summary>
<p>Asegúrate de revisar el enlace para la descripción de information_schema.columns. Te dirá los nombres de las columnas que debes usar para la consulta. Recuerda, queremos el nombre de la tabla, el nombre de la columna y el tipo de datos - ¡revisa la descripción para encontrar los nombres apropiados!</p>
</details>

<details>
<summary>Pista 3</summary>
<p>No olvides usar la declaración ORDER BY para ordenar los resultados de tu búsqueda. Estamos buscando ordenar ambas columnas en orden ascendente, lo cual se puede especificar usando la palabra clave ASC, sin embargo, también es el orden de clasificación predeterminado, por lo que ASC no es necesariamente requerido.</p>
</details>

In [ ]:
%%sql
SELECT * FROM information_schema.columns
ORDER BY table_name ASC, column_name ASC;


,table_catalog,table_schema,table_name,column_name,ordinal_position,column_default,is_nullable,data_type,character_maximum_length,character_octet_length,numeric_precision,numeric_precision_radix,numeric_scale,datetime_precision,interval_type,interval_precision,character_set_catalog,character_set_schema,character_set_name,collation_catalog,collation_schema,collation_name,domain_catalog,domain_schema,domain_name,udt_catalog,udt_schema,udt_name,scope_catalog,scope_schema,scope_name,maximum_cardinality,dtd_identifier,is_self_referencing,is_identity,identity_generation,identity_start,identity_increment,identity_maximum,identity_minimum,identity_cycle,is_generated,generation_expression,is_updatable,COLUMN_COMMENT
0,memory,main,crime_scene_report,city,4,None,YES,VARCHAR,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,NaN,NaN,None,None,None,None,None,NaN,None,None,NaN,None
1,memory,main,crime_scene_report,date,1,None,YES,INTEGER,NaN,NaN,32.0,2.0,0.0,NaN,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,NaN,NaN,None,None,None,None,None,NaN,None,None,NaN,None
2,memory,main,crime_scene_report,description,3,None,YES,VARCHAR,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,NaN,NaN,None,None,None,None,None,NaN,None,None,NaN,None
3,memory,main,crime_scene_report,type,2,None,YES,VARCHAR,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,NaN,NaN,None,None,None,None,None,NaN,None,None,NaN,None
4,memory,main,drivers_license,age,2,None,YES,INTEGER,NaN,NaN,32.0,2.0,0.0,NaN,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,NaN,NaN,None,None,None,None,None,NaN,None,None,NaN,None
5,memory,main,drivers_license,car_make,8,None,YES,VARCHAR,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,NaN,NaN,None,None,None,None,None,NaN,None,None,NaN,None
6,memory,main,drivers_license,car_model,9,None,YES,VARCHAR,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,NaN,NaN,None,None,None,None,None,NaN,None,None,NaN,None
7,memory,main,drivers_license,eye_color,4,None,YES,VARCHAR,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,NaN,NaN,None,None,None,None,None,NaN,None,None,NaN,None
8,memory,main,drivers_license,gender,6,None,YES,VARCHAR,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,NaN,NaN,None,None,None,None,None,NaN,None,None,NaN,None
9,memory,main,drivers_license,hair_color,5,None,YES,VARCHAR,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,NaN,NaN,None,None,None,None,None,NaN,None,None,NaN,None


### Entity Relationship Diagram (Optional)

After doing some detective work and finding the tables in the Police Department database, you discover that there is an ERD! This can be really helpful in your quest to find the murderer. If you like, challenge yourself (and your SQL skills) to continue on by just querying the `information_schema` as needed. Otherwise, you can click the dropdown below to reveal the ERD to help you see the tables and relationships in the Police Department database at a glance:

<details>
  <summary>Check the SQL Murder Mystery ERD</summary>
  <img src="https://github.com/freestackinitiative/coop_sql_notebooks/blob/main/assets/murder_mystery_schema.png?raw=1" alt="ERD for SQL Murder Mystery"/>
</details>

### Diagrama de Relaciones de Entidad (Opcional)

Después de hacer un poco de trabajo de detective y encontrar las tablas en la base de datos del Departamento de Policía, descubres que hay un ERD (Diagrama de Relaciones de Entidad). Esto puede ser realmente útil en tu búsqueda para encontrar al asesino. Si lo prefieres, desafíate a ti mismo (y tus habilidades en SQL) a continuar solo consultando el `information_schema` según sea necesario. De lo contrario, puedes hacer clic en el desplegable a continuación para revelar el ERD que te ayudará a ver las tablas y las relaciones en la base de datos del Departamento de Policía de un vistazo:

<details>
  <summary>Revisa el ERD del SQL Murder Mystery</summary>
  <img src="https://github.com/freestackinitiative/coop_sql_notebooks/blob/main/assets/murder_mystery_schema.png?raw=1" alt="ERD for SQL Murder Mystery"/>
</details>

___

In [9]:
check_suspect("Miranda Priestly")

Congrats, you found the brains behind the murder!
Everyone in SQL City hails you as the greatest SQL detective of all time.
Time to break out the champagne!


True

---

## Resolviendo el Misterio (y verificando si resolviste el caso)

De aquí en adelante, usarás tus habilidades y las pistas en la base de datos del Departamento de Policía para encontrar al asesino. Cuando creas que has encontrado al asesino, vuelve a esta sección y usa la celda de abajo para verificar tu respuesta y ver si encontraste al asesino:

ejemplo:
```python
check_suspect("Adam Levine")
```

## Espacio de Trabajo

Usa las celdas de abajo para escribir tus consultas y trabajar en resolver el misterio. Cuando tengas un sospechoso, verifica tu respuesta usando la [celda arriba de esta](#solving-the-mystery-and-checking-if-you-cracked-the-case) y ejecuta su nombre a través de la función `check_suspect`. Si encuentras al asesino, la función te lo dirá.

No olvides usar `%%sql` en la parte superior de las celdas de `Code` que crees para que tus consultas SQL funcionen. ¡Buena suerte, detectives!

**El crimen fue un `murder` (asesinato) que ocurrió en algún momento `el 15 de Enero de 2018` y que tuvo lugar en `SQL City`.**

In [ ]:
%%sql
--  15 de Enero de 2018 y que tuvo lugar en SQL City
SELECT *
FROM crime_scene_report
WHERE type ='murder'
AND date = 20180115
AND city = 'SQL City'


,date,type,description,city
0,20180115,murder,Security footage shows that there were 2 witnesses. The first witness lives at the last house o...,SQL City


**Seguridad muestra 2 testigos**

*El primer testigo vive en la última casa de "Northwestern Dr"*

In [ ]:
%%sql
SELECT *
FROM person
WHERE address_street_name = 'Northwestern Dr'
ORDER BY address_number DESC
LIMIT 1

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949




*La segunda testigo, llamada Annabel, vive en algún lugar de "Franklin Ave".*


In [ ]:
%%sql
-- Seguridad muestra 2 testigos.
-- La segunda testigo, llamada Annabel, vive en algún lugar de "Franklin Ave".
SELECT *
FROM person
WHERE address_street_name = 'Franklin Ave'
AND name LIKE 'Annabel%'

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


**Los dos testigos fueron entrevistados**

In [ ]:
%%sql
SELECT p.name, i.person_id, i.transcript
FROM interview i JOIN person p ON i.person_id = p.id
WHERE i.person_id IN (14887,16371)

,name,person_id,transcript
0,Morty Schapiro,14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership nu..."
1,Annabel Miller,16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last we..."


**Morty Schapiro :**
Escuché un disparo y luego vi a un hombre salir corriendo. Tenía una bolsa de "Get Fit Now Gym". El número de membresía en la bolsa comenzaba con "48Z". Sólo los miembros de oro tienen esas bolsas. El hombre se subió a un coche con una placa que decía "H42W".

**Annabel Miller :**
Vi ocurrir el asesinato y reconocí al asesino en mi gimnasio cuando estaba haciendo ejercicio la semana pasada, el 9 de enero.

In [ ]:
%%sql
SELECT p.*, ci.check_in_date , gf.*
FROM drivers_license dl
JOIN person p ON dl.id = p.license_id
JOIN get_fit_now_member gf ON p.id = gf.person_id
JOIN get_fit_now_check_in ci ON gf.id = ci.membership_id
WHERE plate_number LIKE '%H42W%'
AND membership_status= 'gold'
AND check_in_date = 20180109;

,id,name,license_id,address_number,address_street_name,ssn,check_in_date,id_1,person_id,name_1,membership_start_date,membership_status
0,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279,20180109,48Z55,67318,Jeremy Bowers,20160101,gold


**Consultar la transcripción de la entrevista del asesino*(Jeremy Bowers)* para encontrar al verdadero villano detrás de este crimen.**

In [ ]:
%%sql
SELECT *
FROM interview
WHERE person_id IN (67318)

,person_id,transcript
0,67318,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" ..."


**Jeremy Bowers:**
Me contrató una mujer con mucho dinero. No sé su nombre, pero sé que mide alrededor de 5'5" (65") o 5'7" (67"). Tiene el pelo rojo y conduce un Tesla Model S. Sé que asistió al Concierto Sinfónico SQL tres veces en diciembre de 2017.

In [ ]:
%%sql
SELECT p.name, dl.*, fb.date, fb.event_name
FROM drivers_license dl
JOIN person p ON dl.id= p.license_id
JOIN facebook_event_checkin fb ON fb.person_id = p.id
WHERE hair_color = 'red'
AND height BETWEEN  65 AND 67
AND car_make = 'Tesla'
AND car_model = 'Model S'
AND gender = 'female'

,name,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model,date,event_name
0,Miranda Priestly,202298,68,66,green,red,female,500123,Tesla,Model S,20171206,SQL Symphony Concert
1,Miranda Priestly,202298,68,66,green,red,female,500123,Tesla,Model S,20171212,SQL Symphony Concert
2,Miranda Priestly,202298,68,66,green,red,female,500123,Tesla,Model S,20171229,SQL Symphony Concert


Según la entrevista que tuvieron con Jeremy Bowers,**Miranda Priestly** seria la mujer que lo contrato para cometer el crimen